In [1]:
import torch
print(torch.cuda.is_available() , torch.__version__)

False 2.2.0+cpu


In [2]:
from src import API
API.Trainer.main(stage = 0 , resume = 0 , checkname= 1)

24-05-30 15:53:11|MOD:display     |: Model Specifics:
24-05-30 15:53:11|MOD:display     |: Start Process [Data] at Thu May 30 15:53:11 2024!


--Process Queue : Data + Fit + Test
--Model_name is set to tra_day_ShortTest!
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=10) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=10,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100, 1]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=10) , display batch progress bar
Callback : StatusDisplay(verbosity=10) , display epoch and event information
Callback : SpecCB_TRA() , in TRA fill [y] [hist_loss] in batch_data.kwargs , update hist_loss in data.buffer
{'random_seed': None,
 'model_name': 'tr

24-05-30 15:53:14|MOD:display     |: Finish Process [Data], Cost 2.4 Secs
24-05-30 15:53:14|MOD:display     |: Start Process [Fit] at Thu May 30 15:53:14 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


Train Ep#  0 loss : 1.01238:   2%|▏         | 1/48 [00:00<00:08,  5.30it/s]

loss function of [pearson] calculated and success!


Valid Ep#  0 score : 0.00335: 100%|██████████| 12/12 [00:00<00:00, 13.85it/s] 
FirstBite Ep#  0 : loss  1.01859, train-0.00283, valid-0.00935, best-0.0093, lr1.3e-03
Valid Ep#  1 score : -0.04865: 100%|██████████| 12/12 [00:00<00:00, 14.02it/s]
FirstBite Ep#  1 : loss  1.01040, train 0.01414, valid 0.03684, best 0.0368, lr2.5e-03
Valid Ep#  2 score : -0.07583: 100%|██████████| 12/12 [00:00<00:00, 12.81it/s]
FirstBite Ep#  2 : loss  0.97749, train 0.04800, valid 0.04607, best 0.0461, lr3.8e-03
Valid Ep#  3 score : -0.03751: 100%|██████████| 12/12 [00:01<00:00, 11.88it/s]
FirstBite Ep#  3 : loss  0.96914, train 0.05821, valid 0.05322, best 0.0532, lr5.0e-03
24-05-30 15:54:02|MOD:display     |: tra_day_ShortTest #0 @20170103|FirstBite Ep#  4 Max Epoch|Train 0.0582 Valid 0.0532 BestVal 0.0532|Cost  0.8Min,  9.4Sec/Ep
24-05-30 15:54:02|MOD:display     |: Finish Process [Fit], Cost 0.0 Hours, 0.8 Min/model, 12.0 Sec/Epoch
24-05-30 15:54:02|MOD:display     |: Start Process [Test] at Thu May 3

                    hook_name  num_calls  total_time  avg_time
10             on_train_batch        192   40.174657  0.209243
27              on_test_batch        282   18.079767  0.064113
15        on_validation_batch         48    2.983947  0.062166
24        on_test_model_start          1    1.715701  1.715701
4          on_fit_model_start          1    1.159580  1.159580
21           on_fit_model_end          1    0.041323  0.041323
17    on_validation_epoch_end          4    0.028157  0.007039
25   on_test_model_type_start          3    0.014962  0.004987
12         on_train_epoch_end          4    0.006974  0.001744
29     on_test_model_type_end          3    0.005334  0.001778
26        on_test_batch_start        282    0.002002  0.000007
28          on_test_batch_end        282    0.001994  0.000007
7        on_train_batch_start        192    0.001175  0.000006
8          on_before_backward        192    0.000999  0.000005
9           on_after_backward        192    0.000998  0

In [7]:
from src import API
API.DataAPI.reconstruct_train_data()

Use device name: NVIDIA GeForce RTX 4090
predict is False , Data Processing start!
4 datas :['y', 'day', '30m', 'risk']
y blocks loading start!
 --> labels blocks reading [ret10_lag] DataBase...... finished! Cost 18.86 secs
 --> labels blocks reading [ret20_lag] DataBase...... finished! Cost 20.08 secs
 --> labels blocks merging (2)...... finished! Cost 2.88 secs
 --> models blocks reading [risk_exp] DataBase...... finished! Cost 67.78 secs
y blocks loading finished! Cost 120.52 secs
y blocks process...... finished! Cost 45.61 secs
y blocks masking...... finished! Cost 0.81 secs
y blocks saving ...... finished! Cost 3.76 secs
y blocks norming...... finished! Cost 0.00 secs
y finished! Cost 170.98 Seconds
--------------------------------------------------------------------------------
day blocks loading start!
 --> trade blocks reading [day] DataBase...... finished! Cost 28.95 secs
day blocks loading finished! Cost 28.98 secs
day blocks process...... finished! Cost 3.27 secs
day blocks 